In [2]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing, manifold, ensemble
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

%matplotlib inline

In [3]:
train_dat = pd.read_csv('features/zeros_ones_train.csv')
test_dat = pd.read_csv('features/zeros_ones_test.csv')
train_labels = np.load('features/train_labels.npy')
train_dat.head()

,Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,CoverageField1B,CoverageField2A,...,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64,Month,Year,Weekday
0,1,23,0.9403,0.0006,7,1.0200,0,17,23,17,...,8,-1,18,-1,10,1,0,8,2013,4
1,5,7,1.0006,0.0040,4,1.2433,0,6,8,6,...,11,-1,17,-1,20,1,2,4,2014,1
2,5,7,1.0006,0.0040,4,1.2433,0,7,12,7,...,21,-1,11,-1,8,1,2,8,2014,0
3,6,10,0.9769,0.0004,1,1.2665,0,3,2,3,...,10,-1,9,-1,21,1,3,4,2013,0
4,4,23,0.9472,0.0006,3,1.3045,0,8,13,8,...,10,-1,11,-1,12,1,1,1,2014,5


In [ ]:
base = xgb.XGBClassifier(nthread=4)

clf = ensemble.BaggingClassifier(base, n_estimators=10, max_samples=1.0, max_features=1.0)
N = int(0.5*len(train_labels))
clf.fit(train_dat.iloc[0:N], train_labels[0:N])

In [ ]:
clf.score(train_dat.iloc[N:], train_labels[N:])

# Bagging

In [5]:
bags = 50
bag_ratio = 0.85
models = []
N = int(0.9*len(train_dat))

xg_test = xgb.DMatrix( train_dat.iloc[N:].as_matrix(), label=train_labels[N:])

X = train_dat.iloc[:N]
Y = train_labels[:N]

param = {   'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'nthread':4,
            'max_depth':6,
            'colsample_bytree':0.77,
            'subsample':0.85,
            'min_child_weight':4,
            'eta':0.02,
        }
num_round = 1000

for i in range(bags):
    print('\rBag {} of {}'.format(i+1,bags),end='')
    
    r = np.random.rand(X.shape[0])
    X_bag = X[r<bag_ratio]
    Y_bag = Y[r<bag_ratio]
    xg_train = xgb.DMatrix( X_bag.as_matrix(), label=Y_bag)
    
    watchlist = [(xg_train,'train'),(xg_test,'eval')]
    model = xgb.train(param, xg_train, num_round, watchlist,
                      early_stopping_rounds=100)
    model.save_model('bag_models/model{}'.format(i+49))
    models.append(model)
    del xg_train

Will train until train error hasn't decreased in 100 rounds.
[0]	eval-auc:0.905343	train-auc:0.903531
[1]	eval-auc:0.935870	train-auc:0.934311
[2]	eval-auc:0.937232	train-auc:0.935070
[3]	eval-auc:0.939361	train-auc:0.936913
[4]	eval-auc:0.940755	train-auc:0.938322
[5]	eval-auc:0.941774	train-auc:0.939605
[6]	eval-auc:0.941790	train-auc:0.939567
[7]	eval-auc:0.942111	train-auc:0.940042
[8]	eval-auc:0.942891	train-auc:0.940879
[9]	eval-auc:0.943295	train-auc:0.941058
[10]	eval-auc:0.943547	train-auc:0.941161
[11]	eval-auc:0.945002	train-auc:0.942518
[12]	eval-auc:0.945938	train-auc:0.943730
[13]	eval-auc:0.945838	train-auc:0.943702
[14]	eval-auc:0.945830	train-auc:0.943718
[15]	eval-auc:0.946022	train-auc:0.943961
[16]	eval-auc:0.946070	train-auc:0.943945
[17]	eval-auc:0.945991	train-auc:0.943832
[18]	eval-auc:0.946135	train-auc:0.943979
[19]	eval-auc:0.946338	train-auc:0.944152
[20]	eval-auc:0.946342	train-auc:0.944192
[21]	eval-auc:0.946562	train-auc:0.944345
[22]	eval-auc:0.946510	tr

Bag 2 of 50

[999]	eval-auc:0.967381	train-auc:0.976473
Will train until train error hasn't decreased in 100 rounds.
[0]	eval-auc:0.921657	train-auc:0.917875
[1]	eval-auc:0.933326	train-auc:0.931086
[2]	eval-auc:0.938252	train-auc:0.935874
[3]	eval-auc:0.942612	train-auc:0.940130
[4]	eval-auc:0.942561	train-auc:0.940059
[5]	eval-auc:0.943574	train-auc:0.941102
[6]	eval-auc:0.945107	train-auc:0.942662
[7]	eval-auc:0.946806	train-auc:0.944325
[8]	eval-auc:0.946108	train-auc:0.943730
[9]	eval-auc:0.945370	train-auc:0.943097
[10]	eval-auc:0.945098	train-auc:0.942944
[11]	eval-auc:0.944896	train-auc:0.942748
[12]	eval-auc:0.945553	train-auc:0.943355
[13]	eval-auc:0.945888	train-auc:0.943552
[14]	eval-auc:0.945767	train-auc:0.943358
[15]	eval-auc:0.946341	train-auc:0.943970
[16]	eval-auc:0.946207	train-auc:0.943878
[17]	eval-auc:0.946036	train-auc:0.943740
[18]	eval-auc:0.946109	train-auc:0.943779
[19]	eval-auc:0.946334	train-auc:0.943968
[20]	eval-auc:0.946889	train-auc:0.944421
[21]	eval-auc:0.947274	t

Bag 3 of 50

Will train until train error hasn't decreased in 100 rounds.
[0]	eval-auc:0.906400	train-auc:0.903903
[1]	eval-auc:0.939062	train-auc:0.936751
[2]	eval-auc:0.942419	train-auc:0.940230
[3]	eval-auc:0.938614	train-auc:0.935987
[4]	eval-auc:0.944659	train-auc:0.942318
[5]	eval-auc:0.943356	train-auc:0.941236
[6]	eval-auc:0.940752	train-auc:0.938679
[7]	eval-auc:0.943206	train-auc:0.941381
[8]	eval-auc:0.942528	train-auc:0.940253
[9]	eval-auc:0.940895	train-auc:0.938561
[10]	eval-auc:0.940205	train-auc:0.937880
[11]	eval-auc:0.942803	train-auc:0.940616
[12]	eval-auc:0.942513	train-auc:0.940146
[13]	eval-auc:0.944193	train-auc:0.941849
[14]	eval-auc:0.944612	train-auc:0.942376
[15]	eval-auc:0.945789	train-auc:0.943523
[16]	eval-auc:0.946087	train-auc:0.943889
[17]	eval-auc:0.946256	train-auc:0.944142
[18]	eval-auc:0.946594	train-auc:0.944521
[19]	eval-auc:0.946812	train-auc:0.944724
[20]	eval-auc:0.947079	train-auc:0.944940
[21]	eval-auc:0.946905	train-auc:0.944763
[22]	eval-auc:0.947001	tr

...

Bag 13 of 50

Will train until train error hasn't decreased in 100 rounds.
[0]	eval-auc:0.928726	train-auc:0.925819
[1]	eval-auc:0.933569	train-auc:0.930917
[2]	eval-auc:0.933355	train-auc:0.930719
[3]	eval-auc:0.933107	train-auc:0.930525
[4]	eval-auc:0.936123	train-auc:0.934305
[5]	eval-auc:0.936172	train-auc:0.934303
[6]	eval-auc:0.936126	train-auc:0.934288
[7]	eval-auc:0.941350	train-auc:0.938673
[8]	eval-auc:0.941444	train-auc:0.938948
[9]	eval-auc:0.941347	train-auc:0.938788
[10]	eval-auc:0.941293	train-auc:0.938745
[11]	eval-auc:0.941908	train-auc:0.939473
[12]	eval-auc:0.941705	train-auc:0.939333
[13]	eval-auc:0.942544	train-auc:0.940114
[14]	eval-auc:0.942697	train-auc:0.940260
[15]	eval-auc:0.942345	train-auc:0.940089
[16]	eval-auc:0.942833	train-auc:0.940508
[17]	eval-auc:0.943274	train-auc:0.941067
[18]	eval-auc:0.943331	train-auc:0.941115
[19]	eval-auc:0.943509	train-auc:0.941278
[20]	eval-auc:0.943786	train-auc:0.941450
[21]	eval-auc:0.943699	train-auc:0.941321
[22]	eval-auc:0.943949	tr

Bag 14 of 50

[999]	eval-auc:0.967511	train-auc:0.976718
Will train until train error hasn't decreased in 100 rounds.
[0]	eval-auc:0.929023	train-auc:0.925561
[1]	eval-auc:0.929106	train-auc:0.925733
[2]	eval-auc:0.937082	train-auc:0.934610
[3]	eval-auc:0.938382	train-auc:0.936245
[4]	eval-auc:0.939616	train-auc:0.937236
[5]	eval-auc:0.939507	train-auc:0.937448
[6]	eval-auc:0.938996	train-auc:0.936682
[7]	eval-auc:0.939403	train-auc:0.936956


KeyboardInterrupt: 

Helpful for bagging xgb models: https://github.com/andyh47/higgs

- Model0 is reaching 0.9683+ after 1750 rounds. Should verify this model's score on the LB
- Model0 peaked at 0.968262 after round 1999 (didn't hit early stop of 100)
- Model1 peak around 0.9684 then drops on the test data, maybe overfitting on train set
- Model2 peak around 0.9682
- Model3 peak around 0.9682
- Model4 peak around 0.9682

In [6]:
def predict(models, data):
    predicted = np.empty((data.num_row(),len(models)))
    predicted[:] = np.NAN
    for i,m in enumerate(models):
        print('\rPredicting {} of {}'.format(i+1,len(models)),end='')
        predicted[:,i] = m.predict(data)
    return np.apply_along_axis(np.mean,1,predicted)

In [7]:
xg_train = xgb.DMatrix( X.as_matrix(), label=Y)
p_train = predict(models, xg_train)
p_test = predict(models, xg_test)

Predicting 13 of 13

In [8]:
train_acc = sum(Y == np.round(p_train))/len(p_train)
test_acc = sum(train_labels[N:] == np.round(p_test))/len(p_test)
print(len(models))
print( train_acc )
print( test_acc)

13
0.937271228114
0.927749654855


~0.968 each for 29 models: 0.96746 LB

In [9]:
xg_pred = xgb.DMatrix( test_dat.as_matrix())

pred = predict(models, xg_pred)
sample = pd.read_csv('sample_submission.csv')
sample.QuoteConversion_Flag = pred
sample.to_csv('bagging_submission03.csv', index=False)

Predicting 13 of 13

In [ ]:
xg_pred = xgb.DMatrix( test_dat.as_matrix())

pred = models[1].predict(xg_pred)
sample = pd.read_csv('sample_submission.csv')
sample.QuoteConversion_Flag = pred
sample.to_csv('ensemble_model1.csv', index=False)

In [ ]:
j

In [ ]:
for root, dirs, files in os.walk("/mydir"):
    for file in files:
        if file.endswith(".txt"):
             print(os.path.join(root, file))